In [31]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Question 1 Data

In [32]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/activity_log_raw.csv', nrows=200000)

In [33]:
df.head()

,SID,ACTIVITY_ID,ACTIVITY_TIME,STATUS
0,584,1291,13-APR-15 10.33.42.190000000 PM,S
1,584,1286,13-APR-15 10.33.42.190000000 PM,S
2,584,1285,13-APR-15 10.33.42.190000000 PM,S
3,584,1284,13-APR-15 10.33.42.190000000 PM,S
4,584,1288,13-APR-15 10.33.42.190000000 PM,S


Task-1. Write a function which returns the date range in the dataset (i.e, earliest and oldest). Please call your function and print out the date range.

In [34]:
df['ACTIVITY_TIME'] = pd.to_datetime(df['ACTIVITY_TIME'], format='%d-%b-%y %I.%M.%S.%f %p')

def date_range(df):
  return df['ACTIVITY_TIME'].min(), df['ACTIVITY_TIME'].max()

min_date, max_date = date_range(df)
print(f"Date range: {min_date} to {max_date}")

Date range: 2015-04-13 00:17:23.170000 to 2015-04-13 22:39:00.431000


Task-2. Write a function which returns the year and month  with the largest number of events.


In [35]:
def max_events_year_month(df):

    df['year_month'] = df['ACTIVITY_TIME'].dt.to_period('M')

    events_by_month = df.groupby('year_month').size()

    max_month = events_by_month.idxmax()

    return max_month

max_activity_month = max_events_year_month(df)
print(f"Year and month with the largest number of events: {max_activity_month}")

Year and month with the largest number of events: 2015-04


In [ ]:
import multiprocessing
import pandas as pd


pool = multiprocessing.Pool()


def process_chunk(chunk):

    num_cols = chunk.shape[1]


    converters = {i: lambda x: x.replace('\0', '') if isinstance(x, str) else x for i in range(num_cols)}

    for col, converter in converters.items():
        chunk[chunk.columns[col]] = chunk[chunk.columns[col]].apply(converter)

    return chunk

chunks = pd.read_csv(
    '/content/drive/MyDrive/activity_log_raw.csv',
    chunksize=1000000,
    encoding='latin-1',
    on_bad_lines='skip',
    sep=',',
    engine='python'
)



result = pool.map(process_chunk, chunks)


result_df = pd.concat(result)


pool.close()

In [ ]:
result_df.head()

Task-3. Describe what strategy you used to deal with the large size of this dataset, any challenges you faced and how you resolved them. [No more than 200 words]


we use multiprocessing to answer this question, we use the strategy chunks which consist of devising our loading procedure into small and load all small churches paralleling. the first challenge we face is the error in encoding our data and some invaluable in our full dataset. to avoid this problem we use the appropriate parameter in our dataset. and the second challenge that we have facing is the some parsing error in our dataset. this method is efficiency because the most depend on the processor of our mahcine.

## Question 2 Data

In [37]:
!unzip '/content/drive/MyDrive/hh_data_ml.zip' -d '/content'

Archive:  /content/drive/MyDrive/hh_data_ml.zip
  inflating: /content/hh_data_ml.csv  


In [38]:
df2 = pd.read_csv('/content/hh_data_ml.csv', sep = '|', nrows=200000)


<ipython-input-38-df227fcb967d>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('/content/hh_data_ml.csv', sep = '|', nrows=200000)


In [39]:
df2.head()

,Unnamed: 0,prov_id,reg_id,dist_id,adm4,urban_rural,hh_id,P03,P05,P07M,...,lan_read_write_fr,lan_read_write_en,lan_read_write_other,P21,P22N,P23,P25,P28,P29,change_log
0,0,1,11,111,11101,1,11101101010011066020020002,0,1,10,...,1,2,2,1,3,6,,2,24,Sun Feb 12 11:03:02 2023
1,1,1,11,111,11101,1,11101101010011066020020002,1,2,8,...,1,2,2,1,3,6,,2,28,Sun Feb 12 11:03:02 2023
2,2,1,11,111,11101,1,11101101010011066020020002,2,1,3,...,1,1,1,1,5,1,4,1,,Sun Feb 12 11:03:02 2023
3,3,1,11,111,11101,1,11101101010011066020020002,2,1,9,...,1,2,2,1,5,2,4,1,,Sun Feb 12 11:03:02 2023
4,4,1,11,111,11101,1,11101101010011066020240024,0,1,12,...,1,2,2,1,5,1,4,2,25,Sun Feb 12 11:03:02 2023


task-1. Calculate the SEM for age in the dataset. Please use no less than 100 bootstrap samples for your calculation. Write a function which takes innumber of  bootstrap samples and returns the SEM. Key columns in the dataset. P07M - birth month; P07A- birth year

In [40]:
import pandas as pd
import numpy as np

def calculate_sem_age(df, n_bootstrap_samples):

    current_year = 2025
    df['age'] = current_year - df['P07A']

    df.loc[df['age'] < 0, 'age'] = np.nan

    df = df.dropna(subset=['age'])

    bootstrap_means = []
    for _ in range(n_bootstrap_samples):
        sample = df['age'].sample(n=len(df), replace=True)
        bootstrap_means.append(np.mean(sample))

    sem = np.std(bootstrap_means)
    return sem

In [41]:
sem_age_100 = calculate_sem_age(df2, 100)
print(f"SEM for age with 100 bootstrap samples: {sem_age_100}")

sem_age_50 = calculate_sem_age(df2, 50)
print(f"SEM for age with 100 bootstrap samples: {sem_age_50}")

sem_age_90 = calculate_sem_age(df2, 90)
print(f"SEM for age with 100 bootstrap samples: {sem_age_90}")

sem_age_80 = calculate_sem_age(df2, 80)
print(f"SEM for age with 100 bootstrap samples: {sem_age_80}")

SEM for age with 100 bootstrap samples: 0.044100219926536455
SEM for age with 100 bootstrap samples: 0.046905638751922184
SEM for age with 100 bootstrap samples: 0.03635456902780064
SEM for age with 100 bootstrap samples: 0.03848744967926644


Task-2. Describe what strategy you used to deal with the fact that you have to run many computations for this task as well as the challenges you faced and how you solved those challenges.

### Handling Potential Errors in Age Calculation

In [ ]:
def handle_invalid_ages(df):
    """
    This function handles invalid age values such as negative ages or missing birth year.
    Invalid ages are replaced with NaN, and rows with NaN are dropped.
    """

    df.loc[df['age'] < 0, 'age'] = np.nan

    df = df.dropna(subset=['age'])
    return df

The code now includes a check for and handling of negative ages, replacing them with NaN values. The rows with NaN are then removed. This prevents errors and ensures the calculation of the mean is more robust.

# Question 3

In [42]:
df3 = pd.read_csv('/content/drive/MyDrive/Africa_Cities.csv')

df4 = df3[(df3["CONTINENT"] == "Africa") & (df3["STATUS"] == "National capital")]["CNTRY_NAME"]
df3.head()

,OBJECTID,Join_Count,CITY_NAME,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,PORT_ID,LABEL_FLAG,CONTINENT,SQMI,SQKM
0,1,1,Jamestown,SHN,St. Helena,SH,St. Helena,National and provincial capital,637,7,"Less than 50,000",46690,0,Africa,1.158346e+07,3.000115e+07
1,2,1,Praia,CPV,Cape Verde,CV,Cape Verde,National capital,113364,5,"100,000 to 249,999",45910,0,Africa,1.158346e+07,3.000115e+07
2,3,1,Santa Cruz de Tenerife,ESP-CNR,Canarias,SP,Spain,Provincial capital,222190,5,"100,000 to 249,999",38160,0,Africa,1.158346e+07,3.000115e+07
3,4,1,Las Palmas,ESP-CNR,Canarias,SP,Spain,Other,378495,4,"250,000 to 499,999",38170,0,Africa,1.158346e+07,3.000115e+07
4,5,1,Funchal,PRT-MDR,Madeira,PO,Portugal,Provincial capital,100847,5,"100,000 to 249,999",0,0,Africa,1.158346e+07,3.000115e+07


In [43]:
import requests
import pandas as pd
from datetime import datetime


df3.head()

API_KEY = '09d395359186ed3715a03c9d93b18874'

BASE_URL = 'https://api.openweathermap.org/data/2.5/forecast'


def get_weather_data(city, country):

    params = {
        'q': f'{city},{country}',
        'appid': API_KEY,
        'units': 'metric',
        'cnt': '40'
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()


        target_date = '2025-01-13'
        target_datetime = datetime(2025, 1, 13, 0, 0)


        for forecast in data['list']:
            forecast_time = datetime.utcfromtimestamp(forecast['dt'])
            if forecast_time.date() == target_datetime.date():
                weather_info = {
                    'Country': country,
                    'City': city,
                    'Date': forecast_time.date(),
                    'Time': forecast_time.time(),
                    'Weather_main': forecast['weather'][0]['main'],
                    'Temp': forecast['main']['temp'],
                    'Temp_min': forecast['main']['temp_min'],
                    'Temp_max': forecast['main']['temp_max'],
                    'Humidity': forecast['main']['humidity'],
                    'Clouds': forecast['clouds']['all']
                }
                return weather_info
    else:
        return None


weather_data_list = []

for city in df4:

    country = city
    weather_data = get_weather_data(city, country)
    if weather_data:
        weather_data_list.append(weather_data)


weather_df = pd.DataFrame(weather_data_list)

weather_df.to_csv('weather_data.csv', index=False)
weather_df

,Country,City,Date,Time,Weather_main,Temp,Temp_min,Temp_max,Humidity,Clouds
0,Mauritania,Mauritania,2025-01-13,00:00:00,Clear,17.21,15.71,17.21,13,0
1,Liberia,Liberia,2025-01-13,00:00:00,Clouds,23.71,21.80,23.71,86,95
2,Eritrea,Eritrea,2025-01-13,00:00:00,Clouds,14.10,13.43,14.10,84,33
3,Comoros,Comoros,2025-01-13,00:00:00,Rain,18.86,18.86,18.93,88,100
4,Congo,Congo,2025-01-13,00:00:00,Clouds,25.70,25.13,25.70,62,92
5,Swaziland,Swaziland,2025-01-13,00:00:00,Rain,18.41,17.98,18.41,94,100
6,Botswana,Botswana,2025-01-13,00:00:00,Clouds,24.11,23.88,24.11,78,100
7,Seychelles,Seychelles,2025-01-13,00:00:00,Clouds,26.52,26.32,26.52,75,93
8,Mauritius,Mauritius,2025-01-13,00:00:00,Rain,21.58,21.58,21.98,90,35
